## Imports

In [ ]:
## Imports
import os
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display
import math
import joblib
import random
import copy

from matplotlib import pyplot as plt
import sklearn
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import h5py
import altair as alt

## Set options
# display all columns in pandas dataframe
pd.set_option('display.max_columns',None)
# allow unlimited number of rows in altair datasets
alt.data_transformers.disable_max_rows()


# Set data path here
data_path = 'wind_plant_data.h5'

## Read and Load Data

In [ ]:
## Read in 2D and 3D onehot encoded representations of the data 
with h5py.File(data_path, 'r') as hf:
    full_2d_arr = hf['/One-hot Encoded/2D Representation/Full 2D array'][()]

In [ ]:
## Convert the Full 2D array to a pandas dataframe with column headers for exploratory data analysis

max_num_turbs = 200
column_names = ['layout', 'scenario', 'opt_yaw', 'num_turbines', 'hub_h', 'rotor_d', 
                'wind_dir', 'wind_speed', 'turbulence']
column_template = ['t_{:03d}', 't_X_{:03d}', 't_Y_{:03d}', 't_ws_{:03d}', 't_yaw_{:03d}', 't_power_{:03d}']
for c in column_template:
    for i in range(max_num_turbs):
        column_names.append(c.format(i))

df = pd.DataFrame(data=full_2d_arr, columns=column_names)
print(df.shape)
df.head()

In [ ]:
## Identify features and labels

# Note: t_ws_XXX is provided for completeness but is not used for this model
features = ['num_turbines', 'wind_dir', 'wind_speed', 'turbulence'] \
              + ['t_{:03d}'.format(i) for i in range(max_num_turbs)]\
              + ['t_X_{:03d}'.format(i) for i in range(max_num_turbs)] \
              + ['t_Y_{:03d}'.format(i) for i in range(max_num_turbs)] \
              + ['t_yaw_{:03d}'.format(i) for i in range(max_num_turbs)]
print('Features to train on:\n', features)

labels = ['t_power_{:03d}'.format(i) for i in range(max_num_turbs)]
print('\nLabels to predict:\n', labels)

local_ws = ['t_ws_{:03d}'.format(i) for i in range(max_num_turbs)]

## Exploratory Data Analysis

In [ ]:
# General statistics of the data
df[features+labels].describe()

In [ ]:
# Create histograms for plant level features to visualized distributions
hist_plant_features = [features[1], features[4], features[5]]
num_cols = 4
num_rows = math.ceil(len(hist_plant_features)/num_cols)

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(5*num_cols,5*num_rows))
for idx, ax in enumerate(axes.flatten()):
    if idx >= len(hist_plant_features):
        ax.remove()
        break
    ax.hist(df[hist_plant_features[idx]], edgecolor='black')
    # sns.histplot(df[hist_plant_features[idx]], stat="percent", ax=ax)
    ax.set_xlabel(hist_plant_features[idx])


In [ ]:
# Create histograms for turbine X and Y locations features to visualized distributions
t_X_idx_start = features.index('t_X_000')
t_X_idx_end = features.index('t_X_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_X_features = features[t_X_idx_start:t_X_idx_end+1]

t_Y_idx_start = features.index('t_Y_000')
t_Y_idx_end = features.index('t_Y_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_Y_features = features[t_Y_idx_start:t_Y_idx_end+1]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
axes[0].hist(df[hist_X_features].unstack().dropna(), edgecolor='black')
axes[0].set_xlabel('Turbine X Locations')
axes[1].hist(df[hist_Y_features].unstack().dropna(), edgecolor='black')
axes[1].set_xlabel('Turbine Y Locations')

plt.show()

In [ ]:
# Create histograms for turbine yaw angle features to visualized distributions
t_yaw_idx_start = features.index('t_yaw_000')
t_yaw_idx_end = features.index('t_yaw_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_yaw_features = features[t_yaw_idx_start:t_yaw_idx_end+1]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
axes[0].hist(df[df['opt_yaw'] == 0][hist_yaw_features].unstack().dropna(), edgecolor='black')
axes[0].set_xlabel('Randomized Yaw Angles')
axes[1].hist(df[df['opt_yaw'] == 1][hist_yaw_features].unstack().dropna(), edgecolor='black')
axes[1].set_xlabel('Optimized Yaw Angles')

plt.show()


In [ ]:
# Create histograms for local turbine wind speeds to visualized distributions
t_ws_idx_start = local_ws.index('t_ws_000')
t_ws_idx_end = local_ws.index('t_ws_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_ws_features = local_ws[t_ws_idx_start:t_ws_idx_end+1]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
axes[0].hist(df[df['opt_yaw'] == 0][hist_ws_features].unstack().dropna(), edgecolor='black')
axes[0].set_xlabel('Local Turbine Wind Speeds')
axes[1].hist(df[df['opt_yaw'] == 1][hist_ws_features].unstack().dropna(), edgecolor='black')
axes[1].set_xlabel('Local Turbine Wind Speeds (optimized yaw)')
plt.show()

In [ ]:
# Create histograms for turbine power outputs features to visualized distributions
t_power_idx_start = labels.index('t_power_000')
t_power_idx_end = labels.index('t_power_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_power_features = labels[t_power_idx_start:t_power_idx_end+1]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
axes[0].hist(df[df['opt_yaw'] == 0][hist_power_features].unstack().dropna(), edgecolor='black')
axes[0].set_xlabel('Turbine Power')
axes[1].hist(df[df['opt_yaw'] == 1][hist_power_features].unstack().dropna(), edgecolor='black')
axes[1].set_xlabel('Turbine Power (optimized yaw)')
plt.show()

In [ ]:
# Create scatter plot for visualizing Plant Power vs plant wind speed
df_alt = df[['opt_yaw','wind_speed']]
df_alt['norm_plant_power'] =  df[hist_power_features].sum(axis=1).div(df['num_turbines']*3.4e6)
df_alt['opt_yaw'] = df_alt['opt_yaw'].astype(bool)

df_alt.head()

non_opt = alt.Chart(df_alt[df_alt['opt_yaw']==0]).mark_circle().encode(
    x = alt.X('wind_speed:Q', title='Plant Level Wind Speed'),
    y = alt.Y('norm_plant_power:Q', title='Normalized Plant Power'),
    color = alt.Color('opt_yaw:N', title='Optimized Yaw Angles')
)

opt = alt.Chart(df_alt[df_alt['opt_yaw']==1]).mark_circle(opacity=0.3).encode(
    x = alt.X('wind_speed:Q', title='Plant Level Wind Speed'),
    y = alt.Y('norm_plant_power:Q', title='Normalized Plant Power'),
    color = alt.Color('opt_yaw:N', title='Optimized Yaw Angles')
)

non_opt + opt

In [ ]:
# Create scatterplot to visualize plant layout and turbine power
t_X_idx_start = features.index('t_X_000')
t_X_idx_end = features.index('t_X_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_X_features = features[t_X_idx_start:t_X_idx_end+1]

t_Y_idx_start = features.index('t_Y_000')
t_Y_idx_end = features.index('t_Y_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_Y_features = features[t_Y_idx_start:t_Y_idx_end+1]

t_power_idx_start = labels.index('t_power_000')
t_power_idx_end = labels.index('t_power_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_power_features = labels[t_power_idx_start:t_power_idx_end+1]

ridx = random.randint(0,len(df)-1)

x_locs = df[hist_X_features].iloc[ridx].values.T
y_locs = df[hist_Y_features].iloc[ridx].values.T
t_powers = df[hist_power_features].iloc[ridx].values.T

df_alt2 = pd.DataFrame([x_locs,y_locs,t_powers]).T
df_alt2.columns = ['x_loc','y_loc','t_powers']
df_alt2.dropna(inplace=True)
df_alt2['t_powers'] = df_alt2['t_powers'].div(1e6)

alt.Chart(df_alt2).mark_circle().encode(
    x = alt.X('x_loc:Q', title='X Location'),
    y = alt.Y('y_loc:Q', title='Y Location'),
    color = alt.Color('t_powers:Q', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(format=',.3f'), title='Turbine Power (MW)'),
)


In [ ]:
# Create scatterplot to visualize plant layout and turbine power
t_X_idx_start = features.index('t_X_000')
t_X_idx_end = features.index('t_X_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_X_features = features[t_X_idx_start:t_X_idx_end+1]

t_Y_idx_start = features.index('t_Y_000')
t_Y_idx_end = features.index('t_Y_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_Y_features = features[t_Y_idx_start:t_Y_idx_end+1]

t_power_idx_start = labels.index('t_power_000')
t_power_idx_end = labels.index('t_power_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
hist_power_features = labels[t_power_idx_start:t_power_idx_end+1]

ridx = random.randint(0,len(df)-1)

x_locs = df[hist_X_features].iloc[ridx].values.T
y_locs = df[hist_Y_features].iloc[ridx].values.T
t_powers = df[hist_power_features].iloc[ridx].values.T

df_alt2 = pd.DataFrame([x_locs,y_locs,t_powers]).T
df_alt2.columns = ['x_loc','y_loc','t_powers']
df_alt2.dropna(inplace=True)
df_alt2['t_powers'] = df_alt2['t_powers'].div(1e6)

plt.figure()
plt.scatter(x=df_alt2['x_loc'], y=df_alt2['y_loc'], c=df_alt2['t_powers'], edgecolor='k')
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()
plt.xlim(np.minimum(xlim[0], ylim[0]), np.maximum(xlim[1], ylim[1]))
plt.ylim(np.minimum(xlim[0], ylim[0]), np.maximum(xlim[1], ylim[1]))
plt.gca().set_aspect(1.)
cbar = plt.colorbar()
cbar.set_label('Power (MW)')

plt.show()



## Model Pipeline

### Define Helper Functions

In [ ]:
def load_full_2d_to_df():
    ## Generate column headers list and data type dictionary for instantiating pandas dataframe
    # Generate turbine specific feature column headers ranging between 000-max_num_turbs, = one-hot encoded values for each turbine 
    t_dummy_vars = ['t_{x}'.format(x=str(y).zfill(3)) for y in range(max_num_turbs)]
    t_X_vars = ['t_X_{x}'.format(x=str(y).zfill(3)) for y in range(max_num_turbs)]
    t_Y_vars = ['t_Y_{x}'.format(x=str(y).zfill(3)) for y in range(max_num_turbs)]
    t_yaw_vars = ['t_yaw_{x}'.format(x=str(y).zfill(3)) for y in range(max_num_turbs)]
    t_ws_vars = ['t_ws_{x}'.format(x=str(y).zfill(3)) for y in range(max_num_turbs)]
    t_power_vars = ['t_power_{x}'.format(x=str(y).zfill(3)) for y in range(max_num_turbs)]
    # Define plant level variables
    plant_vars   = ['layout',           # index for LayoutXXX
                    'scenario',         # index for ScenarioXXX
                    'opt_yaw',          # binary if sample is yaw optimized
                    'num_turbines',     # number of turbines in the layout
                    'hub_h',            
                    'rotor_d',
                    'wind_dir',         # wind direction
                    'wind_speed',       
                    'turbulence',]
    # create list to hold all column names
    column_names = []
    column_names.extend(plant_vars)
    column_names.extend(t_dummy_vars)
    column_names.extend(t_X_vars)
    column_names.extend(t_Y_vars)
    column_names.extend(t_yaw_vars)
    column_names.extend(t_ws_vars)
    column_names.extend(t_power_vars)

    # Define datatypes for each column
    plant_vars_type = ['int32',
                    'int32',
                    'int8',
                    'int32',
                    'float32',
                    'float32',
                    'float32',
                    'float32',
                    'float32']
    turb_vars_type = ['float32' for i in range(max_num_turbs)]

    data_type_dict = dict(zip(plant_vars, plant_vars_type))
    data_type_dict.update(dict(zip(t_dummy_vars,['int8' for i in range(max_num_turbs)])))
    data_type_dict.update(dict(zip(t_X_vars,turb_vars_type)))
    data_type_dict.update(dict(zip(t_Y_vars,turb_vars_type)))
    data_type_dict.update(dict(zip(t_yaw_vars,turb_vars_type)))
    data_type_dict.update(dict(zip(t_ws_vars,turb_vars_type)))
    data_type_dict.update(dict(zip(t_power_vars,turb_vars_type)))

    ## Convert the Full 2D array to a pandas dataframe with column headers for exploratory data analysis
    df = pd.DataFrame(data=full_2d_arr, columns=column_names)
    df = df.astype(data_type_dict)
    
    return df

In [ ]:
def split_data(X, y, split=(0.6, 0.2, 0.2), random_seed=888):
    """
    Splits X/y into training, validation, and testing sets based on the
    specified split.

    Parameters
    ----------
    X : np.ndarray
        Collection of sample features to train on
    y : np.ndarray
        Collection of sample labels to be predicted (turbine power)
    split : tuple
        The percentage allocation of training, validation, and test datasets
        Default: (0.6, 0.2, 0.2)
    random_seed : int
        The seed for the random number generator
        Default: 888

    Returns
    -------
    X_train, X_val, X_test, y_train, y_val, y_test : np.ndarrays
        The X and y training, validation, and test datasets 
    """
    # Input checking
    if len(X) != len(y):
        raise ValueError(f"X and y lengths don't match ({len(X)} != {len(y)})")
    if len(split) != 3:
        raise ValueError("Invalid split, expected 3 percentages (training, validation, test)")
    if sum(split) != 1:
        raise ValueError(f"Invalid split {split}, percentages must sum to 1.0")
    if not isinstance(X, np.ndarray) or not isinstance(y, np.ndarray):
        raise TypeError("X and y must be numpy arrays")

    # Initial split for testing data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=split[2], random_state=random_seed, stratify=X[:,0]         # stratify based on # of optimized yaw simulations in each set (first column in X)
    )

    # Further split for validation data
    val_size = split[1] / (1 - split[2])
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=random_seed, stratify=X_train[:,0]   # stratify based on # of optimized yaw simulations in each set (first column in X_train)
    )

    return X_train, X_val, X_test, y_train, y_val, y_test

### Read, Load, and Preprocess Data

In [ ]:
## Read in 2D and 3D onehot encoded representations of the data
with h5py.File(data_path, 'r') as hf:
    full_2d_arr = hf['/One-hot Encoded/2D Representation/Full 2D array'][()]


In [ ]:
# Load the data 
df = load_full_2d_to_df()

# Fill NaN values with zero
df.fillna(0,inplace=True)

# Save feature and label names
features = list(df.columns[2:-2*max_num_turbs])
# print('Features to train on:\n', features)
labels = list(df.columns[-max_num_turbs:])
# print('\nLabels to predict:\n', labels)
local_ws =list(df.columns[-2*max_num_turbs:-max_num_turbs])
# Helpful variables
n_features = len(features)

# Convert dataframe to numpy arrays of data
X = df[features].values
y = df[labels].values

# Split data into train/validation/test sets with default 60/20/20 split
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)

# Concatenate train and val subsets into one set for post hyperparameter tuning training
X_train_val = np.concatenate((X_train, X_val))
y_train_val = np.concatenate((y_train, y_val))

# # Standardize data
# std_scaler = StandardScaler()

# X_train_std = std_scaler.fit(X_train).transform(X_train)
# X_val_std = std_scaler.fit(X_train).transform(X_val)
# X_test_std = std_scaler.fit(X_train).transform(X_test)

# X_train_val_std = std_scaler.fit(X_train_val).transform(X_train_val)
# X_train_val_test_std = std_scaler.fit(X_train_val).transform(X_test)

# Permutate / shuffle turbine specific features within each record of the dataset to increase decision tree generalization
t_idx_start = features.index('t_000')
t_idx_end = features.index('t_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
t_X_idx_start = features.index('t_X_000')
t_X_idx_end = features.index('t_X_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
t_Y_idx_start = features.index('t_Y_000')
t_Y_idx_end = features.index('t_Y_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
t_yaw_idx_start = features.index('t_yaw_000')
t_yaw_idx_end = features.index('t_yaw_{x}'.format(x=str(max_num_turbs-1).zfill(3)))
t_power_idx_start = labels.index('t_power_000')
t_power_idx_end = labels.index('t_power_{x}'.format(x=str(max_num_turbs-1).zfill(3)))

X_permu = copy.deepcopy(X)
rng = np.random.default_rng(seed=888)
X_permu[:,t_idx_start:t_idx_end+1] = rng.permuted(X_permu[:,t_idx_start:t_idx_end+1], axis=1)
rng = np.random.default_rng(seed=888)
X_permu[:,t_X_idx_start:t_X_idx_end+1] = rng.permuted(X_permu[:,t_X_idx_start:t_X_idx_end+1], axis=1)
rng = np.random.default_rng(seed=888)
X_permu[:,t_Y_idx_start:t_Y_idx_end+1] = rng.permuted(X_permu[:,t_Y_idx_start:t_Y_idx_end+1], axis=1)
rng = np.random.default_rng(seed=888)
X_permu[:,t_yaw_idx_start:t_yaw_idx_end+1] = rng.permuted(X_permu[:,t_yaw_idx_start:t_yaw_idx_end+1], axis=1)


y_permu = copy.deepcopy(y)
rng = np.random.default_rng(seed=888)
y_permu[:,t_power_idx_start:t_power_idx_end+1] = rng.permuted(y_permu[:,t_power_idx_start:t_power_idx_end+1], axis=1)

#Split data into train/validation/test sets with default 60/20/20 split
X_permu_train, X_permu_val, X_permu_test, y_permu_train, y_permu_val, y_permu_test = split_data(X_permu, y_permu)

X_permu_train_val = np.concatenate((X_permu_train, X_permu_val))
y_permu_train_val = np.concatenate((y_permu_train, y_permu_val))


### Decision Tree Model

#### Define Helper Functions

In [ ]:
def build_decision_tree(X_fit, y_fit, X_eval, y_eval, criterion='squared_error', max_depth=None, mode='train', print_output=True, save_model=False, model_name=None):
    """
    Automates the process of creating a decision tree regressor, calculating the mean squared error on training and validation data,
    identifying the features used in the model and their importance, and visualizing the decision tree with graphviz 

    Parameters
    ----------
    criterion : str, {'squared_error', 'friedman_mse', 'absolute_error', 'poisson'}, default='squared_error'
        the function to measure the quality of a split in the decision tree regressor.
    max_depth : int, default=None
        The maximum depth of a the decision tree. If None, then nodes are expanded to minimize the criterion or until all leaves contain less than min_samples_split samples.
    mode : str, {'train', 'test'}, default='train'
        Flag to indicate whether the model is being trained (evaluate performance on validation data) or tested (evaluated on test data)

    Returns
    -------
    mse_fit : float
        The mean squared error of the model when predicting the training data. 
    mse_eval : float
        The mean squared error fo the model when predicting the validation data 
    """

    # Instantiate tree classifier
    dt_regr = DecisionTreeRegressor(criterion=criterion, random_state=888, max_depth=max_depth)

    # Fit the model to the training data set
    dt_regr = dt_regr.fit(X_fit, y_fit)

    # Predict the training and validation data 
    y_fit_pred = dt_regr.predict(X_fit)
    y_eval_pred = dt_regr.predict(X_eval)

    # Calculate training and validation MSE
    mse_fit = calculate_MSEs(y_fit, y_fit_pred)
    mse_eval = calculate_MSEs(y_eval, y_eval_pred)

    # Get model R2 scores
    r2_fit = dt_regr.score(X_fit,y_fit)
    r2_eval = dt_regr.score(X_eval,y_eval) 

    # Get model parameters
    dt_depth = dt_regr.get_depth()
    dt_n_leaves = dt_regr.get_n_leaves()
    dt_parameters = dt_regr.get_params() 

    # save model if flagged true:
    if save_model:
        if not os.path.isdir('./saved_decision_tree_models/'):
            os.system('mkdir ./saved_decision_tree_models/')
        dump_path = './saved_decision_tree_models/'+ model_name + '.joblib'
        joblib.dump(dt_regr, dump_path)

    if (mode == 'train' and print_output == True):
        print('Tree parameters:\n', dt_parameters)
        print('Tree depth:', dt_depth)
        print('Number of leaves:', dt_n_leaves)
        print('Training Turbine MSE:\n', mse_fit[0])
        print('Training Avg Turbine MSE:\n', mse_fit[1])
        print('Training Plant MSE:\n', mse_fit[2])
        print('Validation Turbine MSE:\n', mse_eval[0])
        print('Validation Avg Turbine MSE:\n', mse_eval[1])
        print('Validation Plant MSE:\n', mse_eval[2])
        print('R2 score training:', r2_fit)
        print('R2 score training', r2_eval)  
        # Identify and display the features selected by the decision tree model and their importance
        df_dt_feature_importance = pd.DataFrame()
        df_dt_feature_importance['Feature'] = features
        df_dt_feature_importance['Feature Importance'] = dt_regr.feature_importances_
        df_dt_feature_importance = df_dt_feature_importance.sort_values(by=['Feature Importance'], ascending=False)
        df_dt_feature_importance = df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]
        print('Number of features used in decision tree:', len(df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]))
        print('Features used in the model and their importance:')
        display(df_dt_feature_importance)

    if (mode == 'test' and print_output == True):
        print('Tree parameters:\n', dt_parameters)
        print('Tree depth:', dt_depth)
        print('Number of leaves:', dt_n_leaves)
        print('Training Turbine MSE:\n', mse_train_val[0])
        print('Training Avg Turbine MSE:\n', mse_train_val[1])
        print('Training Plant MSE:\n', mse_train_val[2])
        print('Test Turbine MSE:\n', mse_test[0])
        print('Test Avg Turbine MSE:\n', mse_test[1])
        print('Test Plant MSE:\n', mse_test[2])
        print('R2 score training:', r2_train_val)
        print('R2 score test', r2_test)  
        # Identify and display the features selected by the decision tree model and their importance
        df_dt_feature_importance = pd.DataFrame()
        df_dt_feature_importance['Feature'] = features
        df_dt_feature_importance['Feature Importance'] = dt_regr.feature_importances_
        df_dt_feature_importance = df_dt_feature_importance.sort_values(by=['Feature Importance'], ascending=False)
        df_dt_feature_importance = df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]
        print('Number of features used in decision tree:', len(df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]))
        print('Features used in the model and their importance:')
        display(df_dt_feature_importance)
        
    return mse_fit, mse_eval



In [ ]:
def calculate_MSEs(train, pred):
    train_arr = np.array(train)
    pred_arr = np.array(pred)
    # Turbine level power production, individual turbine MSE across all records in dataset
    turb_mse = np.sum(((pred_arr-train_arr)**2), axis=0)/len(train)
    # Average turbine MSE
    avg_turb_mse = np.sum(turb_mse)/len(turb_mse)
    # Plant level power production. Sums all turbine powers to plant level power production and calculate MSE across all records in the dataset
    plant_mse = np.sum((np.sum(pred_arr, axis=1)-np.sum(train_arr, axis=1))**2)/len(train)


    return [turb_mse, avg_turb_mse, plant_mse]

In [ ]:
# Read and load the saved decision tree model from the .joblib file
def get_saved_decision_tree(model_name=None):
    file_path = './saved_decision_tree_models/'+ model_name + '.joblib'
    dt_regr = joblib.load(file_path)

    return dt_regr

#### Baseline Decision Tree (default parameters)

In [ ]:
mse_train, mse_val = build_decision_tree(X_train, y_train, X_val, y_val, save_model=True, model_name='base_dt')

#### Baseline Permuted Decision Tree (default parameters)

In [ ]:
mse_permu_train, mse_permu_val = build_decision_tree(X_permu_train, y_permu_train, X_permu_val, y_permu_val, save_model=True, model_name='base_permu_dt')

In [ ]:
# Load decision tree regressor from .joblib file
dt_regr = get_saved_decision_tree(model_name='base_dt')

#### Tune Hyperparameters

In [ ]:
# Tune max_depth of baseline tree
# parameter sweep of decision tree parameters
dt_hyp_opt_column_names = ['Tree Depth', 'Training Avg Turbine MSE', 'Validation Avg Turbine MSE', 'Training Plant MSE', 'Validation Plant MSE' ]

max_depths = np.arange(25,225,25)
# criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']

# criterion_depth_combos = list(itertools.product(criterion, max_depths))

results_list_mses = []

for depth in max_depths:
    print("==========================")
    print("Hyperparameters")
    print("max_depth =", depth)
    print("==========================")
    mse_train, mse_val = build_decision_tree(X_train, y_train, X_val, y_val, max_depth=depth, print_output=False)
    results_list_mses.append([depth, mse_train[1], mse_val[1], mse_train[2], mse_val[2]])

# df_dt_hyp_opt = pd.DataFrame(data=results_list, columns=dt_hyp_opt_column_names, ignore_index=True)

In [ ]:
# Tune max_depth of baseline permuted tree
# parameter sweep of decision tree parameters
dt_hyp_opt_column_names = ['Tree Depth', 'Training Avg Turbine MSE', 'Validation Avg Turbine MSE', 'Training Plant MSE', 'Validation Plant MSE' ]

max_depths = np.arange(25,225,25)
# criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']

# criterion_depth_combos = list(itertools.product(criterion, max_depths))

results_list_permu_mses = []

for depth in max_depths:
    print("==========================")
    print("Hyperparameters")
    print("max_depth =", depth)
    print("==========================")
    mse_permu_train, mse_permu_val = build_decision_tree(X_permu_train, y_permu_train, X_permu_val, y_permu_val, max_depth=depth, print_output=False)
    results_list_permu_mses.append([depth, mse_permu_train[1], mse_permu_val[1], mse_permu_train[2], mse_permu_val[2]])

# df_dt_hyp_opt = pd.DataFrame(data=results_list, columns=dt_hyp_opt_column_names, ignore_index=True)

In [ ]:
df_dt_hyp_opt = pd.DataFrame(data=results_list_mses, columns=dt_hyp_opt_column_names)

# Overall the decision tree model does not appear to generalize to new unseen data (validation set)

#### Evaluate Decision Tree (trained on X_train)

In [ ]:
# Instantiate tree classifier
dt_regr = DecisionTreeRegressor(criterion='squared_error', random_state=888, max_depth=100)

# Fit the model to the training data set
dt_regr = dt_regr.fit(X_train, y_train)

# save model
dump_path = './saved_decision_tree_models/'
if os.path.isdir(dump_path):
    os.system('mkdir {}'.format(dump_path))
dump_path += ('tuned_dt_train' + '.joblib')
joblib.dump(dt_regr, dump_path)

# Predict the training, validation, and test data 
y_train_pred = dt_regr.predict(X_train)
y_val_pred = dt_regr.predict(X_val)
y_test_pred = dt_regr.predict(X_test)

# Calculate training and validation MSE
mse_train = [mean_squared_error(y_train, y_train_pred, multioutput='raw_values'),
                mean_squared_error(y_train, y_train_pred, multioutput='uniform_average')]
mse_val = [mean_squared_error(y_val, y_val_pred, multioutput='raw_values'),
            mean_squared_error(y_val, y_val_pred, multioutput='uniform_average')]
mse_test = [mean_squared_error(y_test, y_test_pred, multioutput='raw_values'),
            mean_squared_error(y_test, y_test_pred, multioutput='uniform_average')]

print('Tree parameters:\n', dt_regr.get_params())
print('Tree depth:', dt_regr.get_depth())
print('Number of leaves:', dt_regr.get_n_leaves())
print('Training MSE:\n', mse_train)
print('Validation MSE:\n', mse_val)
print('Test MSE:\n', mse_test)
print('R2 score training:', dt_regr.score(X_train,y_train))
print('R2 score validation', dt_regr.score(X_val,y_val))
print('R2 score test', dt_regr.score(X_test,y_test))   
 # Identify and display the features selected by the decision tree model and their importance
df_dt_feature_importance = pd.DataFrame()
df_dt_feature_importance['Feature'] = features
df_dt_feature_importance['Feature Importance'] = dt_regr.feature_importances_
df_dt_feature_importance = df_dt_feature_importance.sort_values(by=['Feature Importance'], ascending=False)
df_dt_feature_importance = df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]
print('Number of features used in decision tree:', len(df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]))
print('Features used in the model and their importance:')
display(df_dt_feature_importance)

#### Evaluate Decision Tree (trained on X_train_val)

In [ ]:
# # Instantiate tree classifier
dt_regr = DecisionTreeRegressor(criterion='squared_error', random_state=888, max_depth=100)

# Fit the model to the training data set
dt_regr = dt_regr.fit(X_train_val, y_train_val)

# save model
dump_path = './saved_decision_tree_models/'+ 'tuned_dt_train_val' + '.joblib'
joblib.dump(dt_regr, dump_path)

# Predict the training, validation, and test data 
y_train_val_pred = dt_regr.predict(X_train_val)
y_test_pred = dt_regr.predict(X_test)

# Calculate training and validation MSE
mse_train_val = [mean_squared_error(y_train_val, y_train_val_pred, multioutput='raw_values'),
                mean_squared_error(y_train_val, y_train_val_pred, multioutput='uniform_average')]
mse_test = [mean_squared_error(y_test, y_test_pred, multioutput='raw_values'),
            mean_squared_error(y_test, y_test_pred, multioutput='uniform_average')]

print('Tree parameters:\n', dt_regr.get_params())
print('Tree depth:', dt_regr.get_depth())
print('Number of leaves:', dt_regr.get_n_leaves())
print('Training+Validation MSE:\n', mse_train_val)
print('Test MSE:\n', mse_test)
print('R2 score training+validation:', dt_regr.score(X_train_val,y_train_val))
print('R2 score test', dt_regr.score(X_test,y_test) )   
 # Identify and display the features selected by the decision tree model and their importance
df_dt_feature_importance = pd.DataFrame()
df_dt_feature_importance['Feature'] = features
df_dt_feature_importance['Feature Importance'] = dt_regr.feature_importances_
df_dt_feature_importance = df_dt_feature_importance.sort_values(by=['Feature Importance'], ascending=False)
df_dt_feature_importance = df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]
print('Number of features used in decision tree:', len(df_dt_feature_importance[df_dt_feature_importance['Feature Importance'] > 0]))
print('Features used in the model and their importance:')
display(df_dt_feature_importance)

## Concluding Remarks

Here we have demonstrated a potential use case of the floris_data.h5 data in an off-the-shelf sklearn decision tree model. As seen above the decision tree models perform poorly and are unsuccessful at generalizing well to new data. The performance seen from the decision tree models is to be expected. There are inherent limitations in how the data is structured and the algorithms used. By one-hot encoding the turbine information we impose an arbitrary ordering in the turbine related variables that is not representative of an actual ordering of turbines in the generated layouts. For example turbine 000 may be at the forefront of inflow and experience no wake effects in one layout 000 and while in layout 499 the turbine is nested deep into a cluster with different inflow conditions and experiences heavy wake effects. Thus, the model will struggle to generalize to new data with different turbine placement. In an effort to overcome this issue we randomly permutated the order of turbine related variables, however model performance did not increase. Additionally, the logic within a decision tree algorithm splits nodes based on minimizing mean squared error of the predictions. Inherently weak learner models such as decision trees will struggle to capture the complex interactions of wake effects between multiple turbines. A nueral network would be better suited to capture those relationships. We put in the effort to spin up this off-the-shelf model to showcase the utility and increase in performance when implementing a graph neural network (GNN) to capture wake effects and predict wind plant power output. The WPGNN model developed by NREL delivers superior accuracy and speed to these off-the-shelf architectures.  